In [1]:
import pyart
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import xarray as xr
import datetime
import matplotlib.dates as mdates
import warnings
import os
dt = datetime.datetime

scriptStrtT = dt.now()

np.set_printoptions(precision=3,suppress=True,threshold=10000)

warnings.filterwarnings("ignore",category=RuntimeWarning)

# %matplotlib inline

/Users/danstechman/anaconda/lib/python3.5/site-packages/xarray/core/formatting.py:16: FutureWarning: The pandas.tslib module is deprecated and will be removed in a future version.
  from pandas.tslib import OutOfBoundsDatetime


## Plot Settings
This is where the most commonly modified settings reside (i.e., flight, plot start/end times, data location, save path and file type, etc.)

In [32]:
flight = 'RF02_20180119'

if flight == 'RF01_20180116':
    startT = '20180115_230000'
    endT = '20180116_054500'
    
if flight == 'RF02_20180119':
#     startT = '20180119_004500'
#     endT = '20180119_064500'
    
    segNum = 11
    
    startT_segs = ['20180119_033500','20180119_034600','20180119_035300','20180119_041400',
                   '20180119_042300','20180119_044500','20180119_050100','20180119_051200',
                  '20180119_052500','20180119_054200','20180119_055500']
    endT_segs = ['20180119_034500','20180119_035200','20180119_040200','20180119_042300',
                 '20180119_043500','20180119_050000','20180119_051100','20180119_051800',
                '20180119_053300','20180119_054800','20180119_063200']
    dur_segs = [10,6,9,9,12,15,10,6,8,6,37]
    
    
    startT = startT_segs[segNum-1]
    endT = endT_segs[segNum-1]
    dur = dur_segs[segNum-1]

fourPanel_5min  = False
fourPanel_15min = False
fourPanel_variable = True

if fourPanel_5min:
    tDelta = datetime.timedelta(minutes=5)
    saveDir = '5min_4panel'
    
if fourPanel_15min:
    tDelta = datetime.timedelta(minutes=15)
    saveDir = '15min_4panel'

if fourPanel_variable:
    tDelta = datetime.timedelta(minutes=dur)
    saveDir = 'variableSegs_4panel'



hcrFile = '/Volumes/SOCRATES_1/HCR/cfradial/moments/' + flight + '_concat-HCR-moments.nc' 

figSavePath = '/Users/danstechman/GoogleDrive/School/Research/SOCRATES/UI_OU_SOCRATES_Group/SOCRATES/Plots/' + flight + '/HCR/' + saveDir + '/'
os.makedirs(figSavePath,exist_ok=True)

fType = 'png'

## Read data
Read in the concatenated HCR variables

In [3]:
hcrData = xr.open_dataset(hcrFile)

time1d = np.asarray(hcrData['time1d'].data,dtype='datetime64[ns]')
time1d_rnd = (pd.to_datetime(time1d)).round('1s').values
time2d = np.asarray(hcrData['time2d'].data,dtype='datetime64[ns]')
gateAlt = hcrData['gateAlt2d'].data
elev = hcrData['elevation'].data

radDwnwrd = np.where(elev < 0.0)

dbz = hcrData['DBZ'].data
vel = hcrData['VEL'].data
width = hcrData['WIDTH'].data
ldr = hcrData['LDR'].data
ncp = hcrData['NCP'].data
snr = hcrData['SNR'].data

# Adjust radial velocities so negative values are always downward
vel[radDwnwrd,:] *= -1

## Mask Variables
Mask variables to be plotted based on a number of (adjustable) criteria.

In [5]:
# create masked versions of variables, starting with any nan's present
dbz_masked_inv = np.ma.masked_invalid(dbz)
vel_masked_inv = np.ma.masked_invalid(vel)
width_masked_inv = np.ma.masked_invalid(width)
ldr_masked_inv = np.ma.masked_invalid(ldr)

# find indices where alt<0
dbz_masked_alt = np.ma.masked_where(gateAlt < 0,dbz_masked_inv)
vel_masked_alt = np.ma.masked_where(gateAlt < 0,vel_masked_inv)
width_masked_alt = np.ma.masked_where(gateAlt < 0,width_masked_inv)
ldr_masked_alt = np.ma.masked_where(gateAlt < 0,ldr_masked_inv)

# mask variables using thresholding on NCP/SNR
# dbz_masked_ncp = np.ma.masked_where(snr < 6.5,dbz_masked_alt)
# vel_masked_ncp = np.ma.masked_where(snr < 6.5,vel_masked_alt)
# width_masked_ncp = np.ma.masked_where(snr < 6.5,width_masked_alt)
# ldr_masked_ncp = np.ma.masked_where(snr < 6.5,ldr_masked_alt)
dbz_masked_ncp = np.ma.masked_where(ncp < 0.2,dbz_masked_alt)
vel_masked_ncp = np.ma.masked_where(ncp < 0.2,vel_masked_alt)
width_masked_ncp = np.ma.masked_where(ncp < 0.2,width_masked_alt)
ldr_masked_ncp = np.ma.masked_where(ncp < 0.2,ldr_masked_alt)

## Data Index ID
Determine which data indices will provide data between the user-defined start and end times

In [33]:
startT_dt = dt.strptime(startT,'%Y%m%d_%H%M%S')
endT_dt = dt.strptime(endT,'%Y%m%d_%H%M%S')

# Find start and end indices most closely matching current frame
tMatchStrt = min(time1d_rnd, key=lambda x: abs(pd.to_datetime(x) - startT_dt))
startT_ix = np.squeeze(np.where(time1d_rnd == tMatchStrt))[0]
tMatchEnd = min(time1d_rnd, key=lambda x: abs(pd.to_datetime(x) - endT_dt))
endT_ix = np.squeeze(np.where(time1d_rnd == tMatchEnd))[-1]

# Get the actual start/end times (from the time variable, not the user-defined start/end)
strtDT_rnd = pd.to_datetime(time1d_rnd[startT_ix])
endDT_rnd = pd.to_datetime(time1d_rnd[endT_ix])

## Create plots

In [34]:
tmpStrtDT = strtDT_rnd
tmpEndDT = strtDT_rnd + tDelta
while tmpEndDT <= (endDT_rnd + datetime.timedelta(minutes=1)):
    
    if tmpStrtDT.day == tmpEndDT.day:
        titleDTstr = '{} - {}-{}'.format(dt.strftime(tmpStrtDT,'%Y%m%d'),dt.strftime(tmpStrtDT,'%H:%M:%S'),
                                         dt.strftime(tmpEndDT,'%H:%M:%S'))
        if tDelta <= datetime.timedelta(minutes=5):
            saveDTstr = '{}_{}-{}'.format(dt.strftime(tmpStrtDT,'%Y%m%d'),dt.strftime(tmpStrtDT,'%H%M%S'),
                                             dt.strftime(tmpEndDT,'%H%M%S'))
        else:
            saveDTstr = '{}_{}-{}'.format(dt.strftime(tmpStrtDT,'%Y%m%d'),dt.strftime(tmpStrtDT,'%H%M'),
                                             dt.strftime(tmpEndDT,'%H%M'))
    else:
        titleDTstr = '{}-{} - {}-{}'.format(dt.strftime(tmpStrtDT,'%Y%m%d'),dt.strftime(tmpStrtDT,'%H:%M:%S'),
                                         dt.strftime(tmpEndDT,'%Y%m%d'),dt.strftime(tmpEndDT,'%H:%M:%S'))
        if tDelta <= datetime.timedelta(minutes=5):
            saveDTstr = '{}_{}-{}_{}'.format(dt.strftime(tmpStrtDT,'%Y%m%d'),dt.strftime(tmpStrtDT,'%H%M'),
                                             dt.strftime(tmpEndDT,'%Y%m%d'),dt.strftime(tmpEndDT,'%H%M%S'))
        else:
            saveDTstr = '{}_{}-{}_{}'.format(dt.strftime(tmpStrtDT,'%Y%m%d'),dt.strftime(tmpStrtDT,'%H%M'),
                                             dt.strftime(tmpEndDT,'%Y%m%d'),dt.strftime(tmpEndDT,'%H%M'))
    print('Plotting {}'.format(titleDTstr))
    
    # Set up nicely formatted date ticks
#     xtick_locator = mdates.AutoDateLocator()
#     xtick_locator.intervald['MINUTELY'] = [0]
    if tDelta <= datetime.timedelta(minutes=5):
        xtick_formatter = mdates.DateFormatter(fmt='%H:%M:%S')
    else:
        xtick_formatter = mdates.DateFormatter(fmt='%H:%M')
    
    # Find start and end indices most closely matching current frame
    tMatchStrt = min(time1d_rnd, key=lambda x: abs(pd.to_datetime(x) - tmpStrtDT))
    tmpStIx = np.squeeze(np.where(time1d_rnd == tMatchStrt))[0]
    tMatchEnd = min(time1d_rnd, key=lambda x: abs(pd.to_datetime(x) - tmpEndDT))
    tmpEndIx = np.squeeze(np.where(time1d_rnd == tMatchEnd))[-1]

    
    # Initialize figure with 4 subplots, sharing x and y axes
    if fourPanel_5min:
        fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, sharex=True, sharey=True,figsize=(16,28))
        axFsz = 24
        caxFsz = 20
        ttlFsz = 28
        tkFsz = 16
        ctkFsz = 16
        caxShrnk = 0.75
        titleAppnd = ''
        saveAppnd = ''
    if fourPanel_15min:
        fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, sharex=True, sharey=True,figsize=(48,28))
        axFsz = 32
        caxFsz = 28
        ttlFsz = 36
        tkFsz = 24
        ctkFsz = 20
        caxShrnk = 0.9
        titleAppnd = ''
        saveAppnd = ''
    if fourPanel_variable:
        fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, sharex=True, sharey=True,figsize=(16*(dur/5.),28))
        axFsz = 32
        caxFsz = 28
        ttlFsz = 36
        tkFsz = 24
        ctkFsz = 20
        caxShrnk = 0.9
        titleAppnd = ' - Seg #' + str(segNum)
        saveAppnd = '_Seg' + str(segNum)

    # Plot reflectivity and create our figure title
    im1 = ax1.pcolormesh(time2d[tmpStIx:tmpEndIx,:],np.flipud(gateAlt[tmpStIx:tmpEndIx,:]/1000),
                         np.flipud(dbz_masked_ncp[tmpStIx:tmpEndIx,:]),
                         vmin=-40,vmax=16,cmap=pyart.graph.cm.LangRainbow12)
    ax1.set_ylim([0,7])
    cax1 = fig.colorbar(im1,ax=ax1,fraction=0.08,shrink=caxShrnk,pad=0.008)
    cax1.set_label('Reflectivity (dBZ)',fontsize=caxFsz)
    cax1.ax.set_yticklabels(cax1.ax.get_yticklabels(), fontsize=ctkFsz)
    ax1.tick_params(axis='both', which='major', labelsize=tkFsz)
    ax1.set_title('SOCRATES - {} UTC{}'.format(titleDTstr,titleAppnd),fontsize=ttlFsz) 
    ax1.grid()


    # Plot radial velocity and label the y-axis
    im2 = ax2.pcolormesh(time2d[tmpStIx:tmpEndIx,:],np.flipud(gateAlt[tmpStIx:tmpEndIx,:]/1000),
                         np.flipud(vel_masked_ncp[tmpStIx:tmpEndIx,:]),
                         vmin=-5,vmax=5,cmap=pyart.graph.cm.Wild25_r)
    ax2.set_ylim([0,7])
    cax2 = fig.colorbar(im2,ax=ax2,fraction=0.08,shrink=caxShrnk,pad=0.008)
    cax2.set_label('Radial Velocity (m/s)',fontsize=caxFsz)
    cax2.ax.set_yticklabels(cax2.ax.get_yticklabels(), fontsize=ctkFsz)
    ax2.tick_params(axis='both', which='major', labelsize=tkFsz)
    ax2.set_ylabel('Altitude (km)',fontsize=axFsz)
    ax2.grid()


    # Plot LDR
    im3 = ax3.pcolormesh(time2d[tmpStIx:tmpEndIx,:],np.flipud(gateAlt[tmpStIx:tmpEndIx,:]/1000),
                         np.flipud(ldr_masked_ncp[tmpStIx:tmpEndIx,:]),
                         vmin=-40,vmax=0,cmap=pyart.graph.cm.NWSRef)
    ax3.set_ylim([0,7])
    cax3 = fig.colorbar(im3,ax=ax3,fraction=0.08,shrink=caxShrnk,pad=0.008)
    cax3.ax.set_yticklabels(cax3.ax.get_yticklabels(), fontsize=ctkFsz)
    cax3.set_label('LDR (dB)',fontsize=caxFsz)
    ax3.tick_params(axis='both', which='major', labelsize=tkFsz)
    ax3.grid()


    # Plot spectral width and label the x-axis (time)
    im4 = ax4.pcolormesh(time2d[tmpStIx:tmpEndIx,:],np.flipud(gateAlt[tmpStIx:tmpEndIx,:]/1000),
                         np.flipud(width_masked_ncp[tmpStIx:tmpEndIx,:]),
                         vmin=0,vmax=3,cmap=pyart.graph.cm.RefDiff)
    cax4 = fig.colorbar(im4,ax=ax4,fraction=0.08,shrink=caxShrnk,pad=0.008)
    cax4.set_label('Spectral Width (m/s)',fontsize=caxFsz,)
    cax4.ax.set_yticklabels(cax4.ax.get_yticklabels(), fontsize=ctkFsz)
    ax4.set_xlabel('Time (UTC)',fontsize=axFsz)
#     ax4.xaxis.set_major_locator(xtick_locator)
    ax4.xaxis.set_major_locator(mdates.MinuteLocator())
    ax4.xaxis.set_major_formatter(xtick_formatter)
    ax4.tick_params(axis='both', which='major', labelsize=tkFsz)
    ax4.grid()


    # Clean up the date format a bit further and remove extra whitespace between subplots
    fig.autofmt_xdate()
    fig.subplots_adjust(hspace=0.08)


    # Save the output figure
    saveStr = '{}HCR-moments_{}{}.{}'.format(figSavePath,saveDTstr,saveAppnd,fType)
    fig.savefig(saveStr,bbox_inches='tight')

#     print('Plot start: {}'.format(dt.strftime(tmpStrtDT,'%H:%M:%S')))
#     print('Plot end: {}'.format(dt.strftime(tmpEndDT,'%H:%M:%S')))
    tmpStrtDT = tmpEndDT
    tmpEndDT += tDelta

print('Script executed in {}'.format(dt.now() - scriptStrtT))

Plotting 20180119 - 05:55:00-06:32:00
Script executed in 0:36:36.457658
